<a href="https://colab.research.google.com/github/JasperLS/data_science_intros/blob/main/MNIST_and_Fashion_MNIST_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST & Fashion MNIST Classification Notebook
This notebook is losely based on the example provided at https://victorzhou.com/blog/keras-neural-network-tutorial/: 

MNIST is publicly available image data set which consists of 28 by 28 pixels image of hand-written single digits. So each image in MNIST is an image of a digit from 0 to 9. MNIST challenge is to develop a machine learning algorithm that can classify these images into 10 classes (0 to 9).

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. 

## Setup

In [1]:
!pip install mnist

In [2]:
import numpy as np
import mnist
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.utils import to_categorical

In [3]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

In [ ]:
test_images.shape
import pandas as pd
print(pd.Series(test_labels).loc[test_labels == 4].sample(4))
from matplotlib import pyplot as plt
plt.imshow(test_images[95])


In [ ]:
# load alternative fashion mnist dataset
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

## Understand the Data

In [15]:
# how many examples do we have?
print(len(train_images))
print(len(train_labels))

60000
60000


In [16]:
train_labels.shape

(60000,)

In [17]:
# what resolution do our images have?
train_images.shape

(60000, 28, 28)

In [44]:
np.random.randn(4,4).reshape((-1,8)).flatten().mean()

-0.06806993018547927

In [ ]:
# let's look at some examples
from matplotlib import pyplot as plt
num = 132
print('Label:',train_labels[num])
plt.imshow(train_images[num])
plt.show()

In [ ]:
# how many do we have of each example?
np.unique(train_labels, return_counts=True)

In [ ]:
# how is our image data represented?
np.unique(train_images)

## Data Preparation

In [21]:
# normalize the images
train_images = (train_images-train_images.mean())/train_images.std()
test_images = (test_images-train_images.mean())/train_images.std()

In [22]:
# flatten the images
train_images = train_images.reshape((-1, 28*28))
test_images = test_images.reshape((-1, 28*28))

In [ ]:
# do we need to transform labels (later)?
print(to_categorical(train_labels)[:2])
(train_labels[:2])

## Get the Model

In [24]:
# build the model
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

# compile the model
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy', 
  metrics=['accuracy'],
)

# let's take a look at our model
model.summary()

# why do we get the specific number of parameters?

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


## Train the Model

In [25]:
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=5,
  batch_size=64,
  validation_data=(test_images, to_categorical(test_labels)),
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 90us/sample - loss: 0.2694 - accuracy: 0.9187 - val_loss: 6.4153 - val_accuracy: 0.9462
Epoch 2/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.1226 - accuracy: 0.9629 - val_loss: 6.6602 - val_accuracy: 0.9460
Epoch 3/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0906 - accuracy: 0.9721 - val_loss: 4.8353 - val_accuracy: 0.9648
Epoch 4/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0723 - accuracy: 0.9772 - val_loss: 6.3853 - val_accuracy: 0.9518
Epoch 5/5
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0615 - accuracy: 0.9807 - val_loss: 7.1102 - val_accuracy: 0.9524


## Evaluate Model

In [28]:
model.evaluate( 
  test_images, 
  to_categorical(test_labels)
)

# # predict on the first 5 test images.
# predictions = model.predict(test_images[:5])

# # print our model's predictions
# print(np.argmax(predictions, axis=1)) 

# # check our predictions against the ground truths
# print(test_labels[:5]) 

10000/10000 [==============================] - 0s 39us/sample - loss: 7.1102 - accuracy: 0.9524


[7.1101998102441835, 0.9524]

In [ ]:
# wouldn't it be nice to track test loss while training?

## Save (and Load) Model

In [ ]:
model.save('mnist_pretrained_model')

In [ ]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("mnist_pretrained_model")

In [ ]:
reconstructed_model.evaluate( 
  test_images, 
  to_categorical(test_labels)
)

# predict on the first 5 test images.
predictions = reconstructed_model.predict(test_images[:5])

# print our model's predictions
print(np.argmax(predictions, axis=1)) 

# check our predictions against the ground truths
print(test_labels[:5]) 

10000/10000 [==============================] - 0s 47us/sample - loss: 0.8841 - accuracy: 0.9418
[7 2 1 0 4]
[7 2 1 0 4]


In [ ]:
reconstructed_model.summary()

## Try Alternative Model incl. CNNs

##### Get Model inlc. CNN

In [ ]:
model = Sequential([
  Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
  MaxPooling2D((2,2)),
  Flatten(),
  Dense(64, activation='relu'),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

# compile the model
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy', 
  metrics=['accuracy'],
)

# let's take a look at our model


##### Train CNN Model

In [ ]:
model.fit(
  train_images.reshape(train_images.shape[0],28,28,1),
  to_categorical(train_labels),
  epochs=5,
  batch_size=64,
  validation_data=(test_images.reshape(test_images.shape[0],28,28,1), to_categorical(test_labels)),
)

##### Evaluate CNN Model

In [ ]:
model.evaluate( 
  test_images, 
  to_categorical(test_labels)
)

# predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# print our model's predictions
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]